In [4]:
import requests
from bs4 import BeautifulSoup
import json
import re

# Get response
url = 'https://www.boligsiden.dk/resultat/1f923c02d4bf4c0ca6b0e7320ee8daee?s=12&sd=false&d=1&p=1&i=10'
r = requests.get(url)
html = r.content

# Soup it!
soup = BeautifulSoup(html, 'html.parser')
scripts = soup.find_all('script')

# Find script tag w. json data
for s in scripts:
    if '__bs_propertylist_result__ ' in str(s):
        script = str(s)

# Locate JSON string (find better solution)
script = script[45:]
script = script[:-16]

# Convert to json
data =json.loads(script)['result']




In [9]:
data['properties']

[{'id': '9f30566e7f4c41ef97b10714d527f2b2',
  'itemTypeName': 'Ejerlejlighed',
  'itemType': 300,
  'propertyLink': '~/salg/776724403',
  'isFavorite': False,
  'rating': {'ratings': {'conditionRating': None,
    'kitchenRating': None,
    'locationRating': None,
    'bathRating': None},
   'averageRating': None,
   'roundAverageRating': None},
  'priceDevelopment': '0%',
  'hasOpenHouse': False,
  'nextOpenHouse': '01. jan. 00:00',
  'nextOpenHouseSignup': False,
  'imageLink300X200': 'https://pic.boligsiden.dk/property/300x200/1/9f30566e7f4c41ef97b10714d527f2b2.jpg',
  'energyMark': 'c',
  'energyMarkLink': 'https://sparenergi.dk/forbruger/vaerktoejer/find-dit-energimaerke?bbrval=210-3621',
  'address': 'Teglgårdsvej 601, st. tv',
  'postal': '3050',
  'city': 'Humlebæk',
  'paymentCash': '975.000',
  'downPayment': '50.000',
  'paymentExpenses': '2.106',
  'paymentGross': '3.983',
  'paymentNet': '3.509',
  'areaResidential': '52',
  'numberOfRooms': '1',
  'areaParcel': '0',
  'sal

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
df  = pd.DataFrame(data['properties'])

In [ ]:
df.head()